# BASE 1719

Information:
* Periodo: 2017-2019
* Objetivo: predicción de pobreza

## 1. Importar datos y librerias

In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

In [2]:
import pandas as pd
import numpy as np
import pickle
import joblib

In [3]:
import variables_nombres as vn

In [4]:
path = r'..\..\input\vl\preprocessed_data\base17_19_anualizada_hogares.dta'
data_original = pd.read_csv( path )

In [5]:
data_original.shape

(34584, 198)

In [6]:
data_original[ 'region' ]

0         1
1         1
2         1
3         1
4         1
         ..
34579    23
34580    23
34581    23
34582    23
34583    23
Name: region, Length: 34584, dtype: int64

In [7]:
data_original

,conglome,vivienda,hogar,p300a_t,p301a_t,p301d_t,p303_t,p306_t,p310_t,p310b1_t,...,p1175_10_t,p1175_11_t,p1175_12_t,p1175_13_t,p1175_14_t,p1175_15_t,t110_t,gasto_monetario_deflactado,gasto_bruto_deflactado,region
0,5001,2,11,4.0,6.0,1.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,0,2,276.111500,368.075100,1
1,5001,23,11,4.0,5.0,1.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,2,1,88.087260,125.934160,1
2,5001,35,11,4.0,3.0,1.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,2,1,64.939300,100.953470,1
3,5001,46,11,4.0,6.0,1.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,2,2,143.894700,286.947780,1
4,5001,67,11,4.0,3.0,1.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,0,2,59.929256,66.743935,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34579,10801,9,11,4.0,9.0,2.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,0,1,294.939800,425.381160,23
34580,10801,19,11,4.0,10.0,2.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,0,1,143.342380,237.430820,23
34581,10801,25,11,4.0,10.0,2.0,1.0,2.0,1.0,2.0,...,0,0,0,0,0,0,1,212.797640,218.342530,23
34582,10801,30,11,4.0,10.0,1.0,2.0,2.0,2.0,2.0,...,0,0,0,0,0,0,1,239.535250,458.516100,23


### 2. Preprocesar datos

In [8]:
# # Imputación de valores del gasto del percentil 1% superior con valores del percentil 99

# percentil_superior = 0.01
# perc = data_original[ 'gasto_deflactado' ].quantile( 1 - percentil_superior )
# data.loc[ data_original [ 'gasto_deflactado' ] > perc, 'gasto_deflactado' ] = perc

In [9]:
next_vars = [ 'gasto_monetario_deflactado', 'gasto_bruto_deflactado', 'region', 'conglome',	'vivienda',	'hogar' ]
data      = data_original.drop(  next_vars , axis = 1 )

### 3. Importar modelos entrenados y predecir

In [10]:
rf_optimal_model_o = joblib.load( r'..\..\output\ejecucion_2\models\base_17_19\base_17_19_xgb_nrs.joblib' )

y_rf_pred_class_o  = rf_optimal_model_o.predict( data )
y_rf_pred_prob_o   = rf_optimal_model_o.predict_proba( data )[ :, 1 ]

KeyError: 255

### 4. Identificar hogares vulnerables

In [ ]:
data_completa = data_original.copy( deep = True )

data_completa[ 'class_pobreza' ] = y_rf_pred_class_o
data_completa[ 'prob_pobreza' ]  = y_rf_pred_prob_o

NameError: name 'y_rf_pred_class_o' is not defined

Se considera distintas amplitudes de caliper: 
* 1: +-   2
* 2: +-   1
* 3: +- 0.5

In [ ]:
limite_min_1 = 0.070
limite_max_1 = 0.110

limite_min_2 = 0.080
limite_max_2 = 0.100

limite_min_3 = 0.085
limite_max_3 = 0.095

hogares_vulnerables_1 = data_completa[ ( data_completa[ 'prob_pobreza' ] >= limite_min_1 ) & data_completa[ ( data_completa[ 'prob_pobreza' ] <= limite_max_1 ) ]
hogares_vulnerables_2 = data_completa[ ( data_completa[ 'prob_pobreza' ] >= limite_min_2 ) & data_completa[ ( data_completa[ 'prob_pobreza' ] <= limite_max_2 ) ]
hogares_vulnerables_3 = data_completa[ ( data_completa[ 'prob_pobreza' ] >= limite_min_3 ) & data_completa[ ( data_completa[ 'prob_pobreza' ] <= limite_max_3 ) ]

### 5. Calcular la línea de vulnerabilidad

In [ ]:
linea_vul_1 = hogares_vulnerables_1[ 'gasto_deflactado' ].mean()
linea_vul_1

In [ ]:
linea_vul_2 = hogares_vulnerables_2[ 'gasto_deflactado' ].mean()
linea_vul_2

In [ ]:
linea_vul_2 = hogares_vulnerables_2[ 'gasto_deflactado' ].mean()
linea_vul_2